In [11]:
import openai
import pandas as pd
import time
from tqdm import tqdm
from dotenv import load_dotenv
import os

import warnings
warnings.filterwarnings("ignore")

load_dotenv()
openai.api_key=os.getenv("OPENAI_KEY")

In [13]:
df = pd.read_csv('../../data/phmrc/phmrc_adult_tokenized.csv')

In [14]:
label_to_score = {
    'aids-tb': 0,
    'communicable': 1,
    'external': 2,
    'maternal': 3, 
    'non-communicable': 4
}

score_to_label = {
    0: 'aids-tb',
    1: 'communicable',
    2: 'external',
    3: 'maternal',
    4: 'non-communicable' 
}

In [15]:
def extract_score(label: str) -> float:
    label = label.lower()
    if label not in label_to_score:
        print(label)
        return 0
    return label_to_score[label]

In [39]:
prompt_template = """
<narrative>
INPUT
</narrative>

<labels>
aids-tb: Patient died resulting from HIV-AIDs or Tuberculosis.
communicable: Patient died from a communicable disease which is defined as 
illnesses that spread from one human to another such as pneumonia, diarrhea 
or dysentery.
external: Patient died from external causes including as accidents like fires,
drowning, road traffic, falls, poisonous animals and violence like suicide, 
homicide, or other injuries.
maternal: Patient died from complications related to pregnancy or childbirth 
including from severe bleeding, sepsis, pre-eclampsia and eclampsia.
non-communicable: Patient died from a non-communicable disease which is defined
as illnesses that cannot be transmitted from one human to another such as cirrhosis,
epilepsy, acute myocardial infarction, copd, renal failure, cancer, diabetes,
stroke, malaria, asthma, or other non-communicable diseases.
</labels>

<options>
aids-tb, 
communicable, 
external, 
maternal, 
non-communicable
</options>


Which label best applies applies to the narrative (aids-tb, communicable, external, maternal, non-communicable)?
Limit your response to one of the options exactly as it appears in the list.
"""

In [37]:
def inspect_narrative(row):
    print('Narrative: ' + df['narrative'][row])
    print('True Label: ' + df['gs_text34'][row])
    print('Broad Category: ' + df['gs_cod'][row])
    print('Embedding Representation: ' + str(label_to_score[df['gs_cod'][row]]))
    print('GPT-Response: ' + response)

In [50]:
inspect_narrative()

Narrative: the deceased had been burnt and had lost mental balance and died within 1.5 hours of the accident.
True Label: fires
Broad Category: external
Embedding Representation: 2
GPT-Response: non-communicable


In [47]:
# test single one
text = df['narrative'][5]

# call GPT-4
response = None
while response is None:
    try:
        response = openai.ChatCompletion.create(
        model="gpt-4-32k",
        messages=[
                {"role": "user", "content": prompt_template.replace("INPUT", text)},
            ],
        temperature=0
        )
    except Exception as e:
        print(e)
        time.sleep(1)

score = extract_score(response["choices"][0]["message"]["content"])
print(response["choices"][0]["message"]["content"])
print(score)

external
2


In [ ]:
# run all narratives
scores = {}
for i, statement in tqdm(statements.iterrows()):
    text = statement['statements_text']
    # call GPT-4
    response = None
    while response is None:
        try:
            response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                    {"role": "user", "content": prompt_template.replace("INPUT", text)},
                ],
            temperature=0
            )
        except Exception as e:
            print(e)
            time.sleep(1)

    score = extract_score(response["choices"][0]["message"]["content"])

    scores[statement["date"]] = score